In [3]:
import json as js
import pandas as pd
import numpy as np
import math

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import csv

from IPython.display import IFrame

sns.set_color_codes()
sns.set_style("white")

# Opening the JSON file of the game: Atlanta Hawks vs Detroit Pistons, October 27 2015.
# To run this script on any other game, just replace this link with the link to the JSON file of that game.
# @var data_file: JSON File object containing our game's JSON file
data_file = open('nba-movement-data-master/data/unzipped/0021500001.json', "r")
data = js.loads(data_file.read())
data.keys()
# Opening the Shot Log file
# @var shot_log: CSV File object containing the Shot Log for the season
shot_log = list(filter(lambda p: data["gameid"] == p[5],
                  csv.reader(open('nba-movement-data-master/data/shots/shots_fixed.csv', "r"))))

# @var events: list containing all the possessions of the game
events = data["events"]

# @var game_logs_headers: list containing the Column labels for the Game Log dataframe
game_logs_headers = [ 'team_no', 'possession_no', 'teamID_A', 'teamID_B', 'timestamps', 'bh_ID', 'bh_x', 'bh_y',
                      'bh_dist_from_ball', 'bh_dist_from_basket', 'bh_angle_from_basket', 'def1_ID', 'def2_ID',
                      'def3_ID', 'def4_ID', 'def5_ID', 'def1_dist_from_bh', 'def2_dist_from_bh', 'def3_dist_from_bh',
                      'def4_dist_from_bh', 'def5_dist_from_bh', 'def1_rel_angle_from_bh', 'def2_rel_angle_from_bh',
                      'def3_rel_angle_from_bh', 'def4_rel_angle_from_bh', 'def5_rel_angle_from_bh', 'def1_trunc_x',
                      'def1_trunc_y', 'def2_trunc_x', 'def2_trunc_y', 'def3_trunc_x', 'def3_trunc_y', 'def4_trunc_x',
                      'def4_trunc_y', 'def5_trunc_x', 'def5_trunc_y', 'shoot_label' ]

# @var home: list containing the home team's information
home = events[0]["home"]
# @var visitor: list containing the visitor team's information
visitor = events[0]["visitor"]

# @var moments_list: list containing all the timeframes of every possession of the game
moments_list = []
for event in events:
    moments_list.append(event["moments"])

from scipy.spatial.distance import euclidean
# Function to calculate euclidean distance between two players
# @param player_a_loc: location of the first player
# @param player_b_loc: location of the second player
# @return: the euclidean distance between the two players
def player_distance(player_a_loc, player_b_loc):
    return euclidean(player_a_loc, player_b_loc)

def travel_dist(player_locations):
    # get the differences for each column
    diff = np.diff(player_locations, axis=0)
    # square the differences and add them,
    # then get the square root of that sum
    distance = np.sqrt((diff ** 2).sum(axis=1))
    # Then return the sum of all the distances
    return distance.sum()

# Function to determine the ball handler for a specific timeframe
# Player closest to the ball is assigned as the ball handler
# @param plyrs: list of all players on-court
# @return ball_handler: Player object of the ball handler
# @return min_dist: distance between the ball and the ball_handler
# @return basket_dist: distance between the basket and the ball_handler
# @return basket_angle: angle between the basket and the ball_handler
def ball_handler_info(plyrs):
    ball_loc = [plyrs[0][2], plyrs[0][3], 0]
    min_dist = float('inf')
    ball_handler = []
    player_loc = []
    for plyr in plyrs:
        if plyr == plyrs[0]: continue
        player_loc = [plyr[2], plyr[3], 0]
        distance = player_distance(ball_loc, player_loc)
        if distance < min_dist:
            min_dist = distance
            ball_handler = plyr
    bh_loc = [ball_handler[2], ball_handler[3], 0]
    basket_loc = [5.35, -25, 0]
    basket_dist = player_distance(bh_loc, basket_loc)
    basket_angle = math.acos((ball_handler[2] - 5.35) / basket_dist)
    return [ball_handler, min_dist, basket_dist, basket_angle]

# Function to extract information of the defender's on-court
# @param ball_h: Player object of the ball handler
# @param plyrs: list of all players on-court
# @return: list containing all 5 defender's IDs, their distance from the ball handler,
#   their angle from the ball handler and their positions on the truncated coordinate plane
def get_def_info(ball_h, plyrs):
    bh_team_id = ball_h[0]
    bh_loc = [ball_h[2], ball_h[3], 0]
    basket_loc = [5.35, -25, 0]
    def_ids = []
    def_dist_from_bh = []
    def_angle_from_bh = []
    def_trunc_pos = []
    for defs in plyrs:
        if defs[0] != -1 and defs[0] != bh_team_id: # Ensure that the player is a defender
            # Add to defender list
            def_ids.append(defs[1])
            # Calculate def's distance from ball handler
            plyr_loc = [defs[2], defs[3], 0]
            def_distance = player_distance(bh_loc, plyr_loc)
            def_dist_from_bh.append(def_distance)
            # Calculate def's relative angle using cosine rule
            a = player_distance(bh_loc, basket_loc)
            b = player_distance(bh_loc, plyr_loc)
            c = player_distance(plyr_loc, basket_loc)
            if b != 0:
                def_angle = math.acos( (c**2 - a**2 - b**2) / (-2*a*b) )
                if ball_h[3] < defs[3]:
                    def_angle = (2 * math.pi) - def_angle
                def_trunc_x = def_distance * math.cos(def_angle)
                def_trunc_y = def_distance * math.sin(def_angle)
            else:
                def_angle = float('inf')
                def_trunc_x = 0
                def_trunc_y = 0
            def_angle_from_bh.append(def_angle)
            # Def's truncated positions
            def_trunc_pos.append(def_trunc_x)
            def_trunc_pos.append(def_trunc_y)
    # Merge info to be returned
    return_list = def_ids                 # Def IDs
    return_list.extend(def_dist_from_bh)  # Def_dist_from_bh
    return_list.extend(def_angle_from_bh) # Def_rel_angle_from_bh
    return_list.extend(def_trunc_pos)     # Def_trunc
    return return_list


# @var possessions: list containing Game Logs of every possession of the game
possessions = []

# This for-loop will loop through each of the game possessions
# @var moments: list of all timeframes of the current possession
for moments in moments_list:
    # @var game_logs: Game Log of the current possession
    game_logs = []
    # @var possession_no: Possession number of the current possession
    possession_no = moments_list.index(moments)
    # @var event_id: ID of the current possession
    event_id = events[possession_no]["eventId"]
    # @var event_shot_log: instance of the shot from the Shot Log
    #   that was attempted in the current possession
    event_shot_log = list(filter(lambda p: event_id == p[4], shot_log))
    # @var shoot_labelled: boolean flag of whether the shot has been labelled
    #   for the current possession
    shoot_labelled = False

    # This for-loop will loop through each timeframe of the current possession
    # @var moment: the current timeframe of the current possession
    for moment in moments:
        # @var bh_info: list containing the current timeframe's ball handler's information
        bh_info = ball_handler_info(moment[5])
        # @var bh: Player object of the ball handler
        bh = bh_info[0] # Ball handler

        # Populating the Game Log with the current timeframe
        game_log = [bh[0],                       # Team no
                    possession_no,               # Possession no
                    home["teamid"],              # Team ID A (Home)
                    visitor["teamid"],           # Team ID B (Visitor)
                    moment[3],                   # Timestamp (Shot clock)
                    bh[1],                       # Ball handler ID
                    bh[2],                       # Ball handler X position
                    bh[3],                       # Ball handler Y position
                    bh_info[1],                  # Ball handler distance from ball
                    bh_info[2],                  # Ball handler distance from basket
                    bh_info[3]]                  # Ball handler angle from basket
        # Add defenders info to game log
        def_info = get_def_info(bh, moment[5])
        game_log.extend(def_info) # Def_ID, Def_dist_from_bh, Def_rel_angle_from_bh, Def_trunc

        # Add shoot label to game log if exists and not already added
        if len(event_shot_log) != 0:
            if not shoot_labelled and float(event_shot_log[0][19]) == moment[2]:
                game_log.append(1)
                shoot_labelled = True
        else:
            game_log.append(0)

        # Append current timeframe to the current possession's Game Log
        game_logs.append(game_log)

    # Append current possession's Game Log to the list of all possession's Game Logs
    possessions.append(game_logs)

# Visualize our Game Logs: the 2nd possession is used for testing purposes.
# The index provided to the possessions can be altered to visualize any other possession's Game Log
df = pd.DataFrame(possessions[1], columns=game_logs_headers)
pd.set_option('display.max_rows', 700)
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 150)
df

,team_no,possession_no,teamID_A,teamID_B,timestamps,bh_ID,bh_x,bh_y,bh_dist_from_ball,bh_dist_from_basket,bh_angle_from_basket,def1_ID,def2_ID,def3_ID,def4_ID,def5_ID,def1_dist_from_bh,def2_dist_from_bh,def3_dist_from_bh,def4_dist_from_bh,def5_dist_from_bh,def1_rel_angle_from_bh,def2_rel_angle_from_bh,def3_rel_angle_from_bh,def4_rel_angle_from_bh,def5_rel_angle_from_bh,def1_trunc_x,def1_trunc_y,def2_trunc_x,def2_trunc_y,def3_trunc_x,def3_trunc_y,def4_trunc_x,def4_trunc_y,def5_trunc_x,def5_trunc_y,shoot_label
0,1610612737,0,1610612737,1610612765,24.00,201143,47.26914,24.47616,1.672006,64.846779,0.867895,101141,202704,202694,203484,203083,9.067459,26.391324,10.880868,9.993431,0.269572,3.894713,3.971149,3.717315,0.822704,1.846539,-6.615244,-6.201400,-17.819508,-19.467077,-9.126866,-5.923984,6.797950,7.325062,-0.073394,0.259388,0
1,1610612737,0,1610612737,1610612765,24.00,201143,47.38016,24.35660,1.892510,64.827527,0.865397,101141,202704,202694,203484,203083,8.960070,26.286936,10.996589,9.868467,0.102676,3.876454,3.964018,3.732179,0.815504,2.051273,-6.647690,-6.007583,-17.886833,-19.263027,-9.133923,-6.123433,6.764847,7.184949,-0.047457,0.091051,0
2,1610612737,0,1610612737,1610612765,24.00,201143,47.44308,24.29835,1.903352,64.824029,0.864075,101141,202704,202694,203484,203083,8.943960,26.225102,11.057963,9.807951,0.000000,3.867562,3.960563,3.741109,0.811180,inf,-6.688797,-5.937543,-17.911058,-19.155939,-9.129551,-6.239380,6.754183,7.111745,0.000000,0.000000,0
3,1610612737,0,1610612737,1610612765,24.00,201143,47.44450,24.29213,1.877423,64.820221,0.863996,101141,202704,202694,203484,203083,8.973570,26.222622,11.065638,9.805580,0.000000,3.866444,3.960482,3.742548,0.811270,inf,-6.717596,-5.949694,-17.910914,-19.152678,-9.126894,-6.256847,6.751910,7.110633,0.000000,0.000000,0
4,1610612737,0,1610612737,1610612765,24.00,201143,47.41923,24.26615,1.926306,64.784054,0.864032,101141,202704,202694,203484,203083,8.994890,26.249015,11.093813,9.791290,0.000000,3.865813,3.959742,3.741960,0.814263,inf,-6.737318,-5.959580,-17.943114,-19.158691,-9.153818,-6.267400,6.720784,7.120423,0.000000,0.000000,0
5,1610612737,0,1610612737,1610612765,24.00,201143,47.41252,24.24539,1.946149,64.763910,0.863903,101141,202704,202694,203484,203083,8.994695,26.256196,11.119252,9.775881,0.000000,3.861024,3.959156,3.743123,0.815455,inf,-6.765634,-5.927120,-17.959260,-19.153402,-9.167495,-6.292440,6.701731,7.117207,0.000000,0.000000,0
6,1610612737,0,1610612737,1610612765,24.00,201143,47.41832,24.25293,1.977594,64.773410,0.863910,101141,202704,202694,203484,203083,8.996171,26.252426,11.115225,9.788367,0.000000,3.861333,3.959623,3.744949,0.815139,inf,-6.764912,-5.930181,-17.947732,-19.159039,-9.152675,-6.306883,6.712544,7.124176,0.000000,0.000000,0
7,1610612737,0,1610612737,1610612765,24.00,201143,47.38687,24.24348,1.950888,64.745801,0.864185,101141,202704,202694,203484,203083,9.036542,26.288880,11.130568,9.784847,0.000000,3.861121,3.959727,3.743146,0.818273,inf,-6.796533,-5.955354,-17.970658,-19.187513,-9.176678,-6.299058,6.687778,7.142608,0.000000,0.000000,0
8,1610612737,0,1610612737,1610612765,24.00,201143,47.38229,24.25949,2.037439,64.755006,0.864399,101141,202704,202694,203484,203083,9.060479,26.298491,11.116753,9.805458,0.000000,3.864958,3.960674,3.743508,0.818554,inf,-6.791574,-5.997234,-17.959054,-19.211533,-9.163016,-6.294548,6.699849,7.159542,0.000000,0.000000,0
9,1610612737,0,1610612737,1610612765,24.00,201143,47.39271,24.24378,2.020844,64.749821,0.864119,101141,202704,202694,203484,203083,9.059575,26.292855,11.130659,9.792124,0.000000,3.861783,3.959968,3.745753,0.818285,inf,-6.809904,-5.975041,-17.968756,-19.194739,-9.160303,-6.323006,6.692665,7.148001,0.000000,0.000000,0
